In [1]:
import pandas as pd, numpy as np, pyodbc, collections, matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def queryDBmonthly(predicate='count(*)',table_name=True,_where='',year=2000,month=1,endyear=2020,endmonth=6,server='<your server_name>',DB='<your DB_name>',dictionary=dict()):
    '''This fucntion connects to the MSSQL server DB database to perform predicate query in monthly fashion.
    endmonth is a parameter that allows you to choose when you want to end your query.
    
    If table_name is False, then this function treats predicate as the entire body of the SQL query.
    In this case, the predicate must be a valid SQL query against any of the tables in the DB.
    Using T-SQL and MSSQL flavours are encouraged, but you should be good with standard SQL.
    
    If you are using an aggregate in your predicate, then your predicate should contain only aggregate functions.
    
    This function cannot handle a mix of aggregate functions and raw table columns.
    If you need this functionality, you need to add the logic to the script manually.
    
    The return value for this function is a dictionary with table_name as the key,
    and list of values from table_name. An additional output is an ordered list of table_names.
    
    If you have more than one column in your resultset, then you need to manually handle it,
    or extend the logic in of this function.
    
    Note that the primary purpose of this function is to prepare one column for charting.'''
    
    key = list() # list of table_names
    value = list() # list of values
    
    if len(dictionary) != 0:
        del dictionary
    dictionary = dict()
    
    if table_name == True:
        condition = False # the condition to end the loop
        while condition == False:
            conn = pyodbc.connect('DRIVER={SQL Server};SERVER=%s;DATABASE=%s' % (server, DB))
            cur = conn.cursor()

            if _where == '':
                _where = '1=1'

            if year == endyear and month == endmonth:
                condition = True
                break
            elif month != 12:
                table_name = 'wt_'+str(year)+'_'+str(month)
                key.append(table_name)
                cur.execute(f'select {predicate} from {table_name} where {_where}')
                dictionary[table_name] = cur.fetchall()
                month += 1
            elif month == 12:
                table_name = 'wt_'+str(year)+'_'+str(month)
                key.append(table_name)
                cur.execute(f'select {predicate} from {table_name} where {_where}')
                dictionary[table_name] = cur.fetchall()
                month = 1
                year += 1

            conn.close()  
    else:
        conn = pyodbc.connect('DRIVER={SQL Server};SERVER=%s;DATABASE=%s' % (server, DB))
        cur = conn.cursor()
        cur.execute(f'{predicate}')
        dictionary['KEY'] = cur.fetchall()
        conn.close()
            
    clean = {k:i for k, v in dictionary.items() for item in v for i in item}
    return [clean, key, value]

In [3]:
def queryDB(query='',server='<your server_name>',DB='<your DB_name>'):
    '''free style querying...
    "query" must be a valid SQL query against any of the tables
    in the DB. Use standard SQL. Do not complicate the query
    beyond the basic SQL clauses.
    
    Instructions:
    1. Put your column name in [] if and only if your column name
    consists of several words with spaces in between.
    
    2. You should also separate column names like this: ", "; the space
    after comma is essential to how this function operates.
    
    3. Do not use the wildcard * for all columns!
    The Purpose of this function is not replace the database engine,
    but to manipulate data for charting. To display all columns or 
    to query them, go back to SQL Server RDBMS.
    
    4. I have not handled all the bugs here. They need to be handled
    on a case-by-case basis. For example, I have not handled casting
    variables.'''
    
    def delComma(s):
        return s.replace(',', '').capitalize()
    
    def delBracket(s):
        return s.replace('[', '').replace(']', '').capitalize()
    
    dictionary=collections.OrderedDict()
    
    if len(dictionary) != 0:
        del dictionary
        dictionary = collections.OrderedDict()
        
    key = list() # list of column names
    
    if query == '':
        warn = print('Warning! "query" must be a valid SQL query.')
        return warn
    else:
        words = query.split(' ')
        colWithSpace = False
        ww = []
        for word in words:
            if word.lower()=='select' or word.lower()=='distinct' or \
            word.lower()=='top' or word.isdigit()==True: pass
            elif word.lower() == 'from': break
            elif word.startswith('['):
                colWithSpace = True
                wordIt = delBracket(word)
                ww.append(wordIt)
            elif (word.lower().startswith('count') or \
            word.lower().startswith('sum') or \
            word.lower().startswith('max') or \
            word.lower().startswith('min')) and \
            '[' in word:
                colWithSpace = True
                wordIt = delBracket(word)
                ww.append(wordIt)
            elif colWithSpace and (word.endswith('])') or \
            word.endswith('],') or word.endswith(']),')):
                colWithSpace = False
                wordIt = delBracket(delComma(word))
                ww.append(wordIt)
                phrase = "".join(['_'+p for p in ww])
                key.append(phrase)
                dictionary[phrase] = list()
                ww = []
                del phrase
            elif colWithSpace and not word.endswith(']'):
                ww.append(word.capitalize())
            else:
                if colWithSpace == False:
                    w = delComma(word.capitalize())
                    key.append(w)
                    dictionary[w] = list()

        conn = pyodbc.connect('DRIVER={SQL Server};SERVER=%s;DATABASE=%s' % (server, DB))
        cur = conn.cursor()
        cur.execute(f'{query}')
        things = cur.fetchall()
        conn.close()
    
    if '' in dictionary.keys():
        del dictionary['']
    for n in range(0, len(dictionary)):
        for item in things:
            dictionary[key[n]].append(item[n])
    return pd.DataFrame(dictionary)